Content-only music recommender that uses STFT-derived features and MFCCs to return nearest neighbors.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

COMPILING AUDIO DATASET USING FMA DATASET

In [ ]:
# FMA-small → WAV mini-dataset builder
# Defaults: N_SAMPLES = 40, SR = 22050, MONO WAV, 30 s clips

!pip -q install pandas==2.2.2 librosa==0.10.2.post1 soundfile==0.12.1

import os, io, zipfile, random, csv, math
import numpy as np
import pandas as pd
import librosa, soundfile as sf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.3 MB/s eta 0:00:00


In [ ]:
# Settings

N_SAMPLES   = 40           # choose any 30–80
SR          = 22050        # target sample rate
CLIP_SEC    = 30           # enforce 30 s
SEED        = 123

OUT_DIR     = "project_dataset"
CLIPS_DIR   = os.path.join(OUT_DIR, "clips")
os.makedirs(CLIPS_DIR, exist_ok=True)

In [ ]:
# FMA-small audio and metadata zips

AUDIO_ZIP_URL = "https://os.unil.cloud.switch.ch/fma/fma_small.zip"
META_ZIP_URL  = "https://os.unil.cloud.switch.ch/fma/fma_metadata.zip"

def wget(url, out):
    import subprocess, shlex, textwrap
    cmd = f"wget -q --show-progress -O {shlex.quote(out)} {shlex.quote(url)}"
    print(f"Downloading: {url}")
    r = subprocess.run(cmd, shell=True)
    if r.returncode != 0:
        raise RuntimeError(f"wget failed for {url}")

In [ ]:
!wget https://os.unil.cloud.switch.ch/fma/fma_small.zip

--2025-10-26 23:30:13--  https://os.unil.cloud.switch.ch/fma/fma_small.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.16, 2001:620:5ca1:201::214
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7679594875 (7.2G) [application/zip]
Saving to: ‘fma_small.zip’

fma_small.zip       100%[===================>]   7.15G  97.8MB/s    in 88s     

2025-10-26 23:31:40 (83.7 MB/s) - ‘fma_small.zip’ saved [7679594875/7679594875]



In [ ]:
# Download missing zips

if not os.path.exists("fma_small.zip"):
    wget(AUDIO_ZIP_URL, "fma_small.zip")
if not os.path.exists("fma_metadata.zip"):
    wget(META_ZIP_URL, "fma_metadata.zip")

Downloading: https://os.unil.cloud.switch.ch/fma/fma_metadata.zip


In [ ]:
# Unzip Audio to ./fma_small/

if not os.path.exists("fma_small"):
    print("Extracting fma_small.zip ...")
    with zipfile.ZipFile("fma_small.zip", "r") as z:
        z.extractall(".")
else:
    print("Audio already extracted.")

Extracting fma_small.zip ...


In [ ]:
# Unzip metadata to ./fma_metadata/

if not os.path.exists("fma_metadata"):
    print("Extracting fma_metadata.zip ...")
    with zipfile.ZipFile("fma_metadata.zip", "r") as z:
        z.extractall(".")
else:
    print("Metadata already extracted.")

Metadata already extracted.


In [ ]:
# tracks.csv is a multi-index column CSV so flatten it for easy access.

tracks_csv_path = "fma_metadata/tracks.csv"
tracks = pd.read_csv(tracks_csv_path, header=[0,1], index_col=0)

# Flatten multiindex columns like ('track','title') → 'track.title'
tracks.columns = ['{}.{}'.format(a,b) for a,b in tracks.columns.values]

In [ ]:
# Keeping only the fields we need

keep_cols = [
    'track.title',
    'track.genre_top',
    'artist.name'
]
missing_cols = [c for c in keep_cols if c not in tracks.columns]
if missing_cols:
    raise ValueError(f"Missing expected columns in tracks.csv: {missing_cols}")

meta = tracks[keep_cols].copy()
meta.rename(columns={
    'track.title': 'title',
    'artist.name': 'artist',
    'track.genre_top': 'genre'
}, inplace=True)

In [ ]:
# Substitute rows may have NaNs with 'unknown'

meta['title']  = meta['title'].fillna('unknown')
meta['artist'] = meta['artist'].fillna('unknown')
meta['genre']  = meta['genre'].fillna('unknown')

In [ ]:
# discover available MP3 files
# Audio files are named by 6-digit track id in nested folders:
# fma_small/000/000002.mp3 for track_id 2, etc.

def id_to_mp3_path(tid: int) -> str:
    tid6 = f"{tid:06d}"
    folder = tid6[:3]
    return os.path.join("fma_small", folder, f"{tid6}.mp3")

available = []
for tid in meta.index.tolist():
    p = id_to_mp3_path(int(tid))
    if os.path.exists(p):
        available.append(int(tid))

print(f"Tracks with audio present: {len(available)}")


Tracks with audio present: 8000


In [ ]:
# Sampling a subset

random.seed(SEED)
if len(available) == 0:
    raise RuntimeError("No audio files found; check that fma_small extracted correctly.")
pick = random.sample(available, k=min(N_SAMPLES, len(available)))
print(f"Sampling {len(pick)} tracks.")

Sampling 40 tracks.


In [ ]:
# Export WAV + build metadata.csv

rows = [("track_id","title","artist","genre","path")]
target_len = SR * CLIP_SEC

for i, tid in enumerate(pick, start=1):
    mp3_path = id_to_mp3_path(tid)
    # Load MP3, resample to SR, mono, duration 30 s
    # librosa.load will trim to duration, but we ensure exact length by pad/truncate
    y, sr = librosa.load(mp3_path, sr=SR, mono=True, duration=CLIP_SEC)
    if len(y) < target_len:
        y = np.pad(y, (0, target_len - len(y)))
    elif len(y) > target_len:
        y = y[:target_len]

    # Write as WAV PCM_16
    out_name = f"{i:03d}.wav"
    out_path = os.path.join(CLIPS_DIR, out_name)
    sf.write(out_path, y.astype(np.float32), SR, subtype="PCM_16")

    # Metadata fields
    m = meta.loc[tid]
    title  = str(m['title'])
    artist = str(m['artist'])
    genre  = str(m['genre']) if pd.notna(m['genre']) else 'unknown'

    rows.append((str(tid), title, artist, genre, f"clips/{out_name}"))


In [ ]:
# Save CSV
meta_csv_path = os.path.join(OUT_DIR, "metadata.csv")
with open(meta_csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(rows)

print("\nDone.")
print(f"Saved WAV clips → {CLIPS_DIR}")
print(f"Metadata CSV   → {meta_csv_path}")


Done.
Saved WAV clips → project_dataset/clips
Metadata CSV   → project_dataset/metadata.csv


In [ ]:
# Peek at first five rows
for r in rows[:10]:
    print(r)

('track_id', 'title', 'artist', 'genre', 'path')
('10440', 'No Gravity Blues', 'Big Blood', 'Folk', 'clips/001.wav')
('53576', 'River', 'Mary Lorson', 'Folk', 'clips/002.wav')
('14653', 'Bad Sign', 'Brad Sucks', 'Rock', 'clips/003.wav')
('126182', 'Funkymeter', 'Joao Picoito', 'Instrumental', 'clips/004.wav')
('73761', 'Weapons', 'Dad Rocks!', 'International', 'clips/005.wav')
('53229', 'Cocaine', 'Fruit Flesh', 'Pop', 'clips/006.wav')
('20667', 'surprisingly upbeat about it', 'Vim', 'Electronic', 'clips/007.wav')
('131940', 'Illede Roman Olsun', 'Choban Elektrik', 'International', 'clips/008.wav')
('140263', 'Dance Of My Imagination', 'Illusory Scapes', 'Electronic', 'clips/009.wav')


Short-Time Fourier Transform (STFT)

Compute STFT magnitude spectrogram |X(n,k)|, with typical parameters: FFT size 2048, hop length 512, Hann window. Derive summary features like spectral centroid, roll-off, flatness, and zero-crossing rate from time frames, then aggregate with mean and standard deviation across time.

In [ ]:
!pip install librosa==0.10.1 soundfile==0.12.1 numpy pandas scikit-learn tqdm --quiet

import os
import math
import json
import random
import warnings
from dataclasses import dataclass

import numpy as np
import pandas as pd
from tqdm import tqdm

import librosa
import librosa.display

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Paths
DATA_DIR = "/content/data"
AUDIO_DIR = os.path.join(DATA_DIR, "audio")
TRACKS_CSV = os.path.join(DATA_DIR, "tracks.csv")
FEATURES_CSV = os.path.join(DATA_DIR, "features.csv")

In [ ]:
# Audio/Feature parameters

SR = 22050
CLIP_SECONDS = 30

N_FFT = 2048
HOP_LENGTH = 512
WIN = "hann"

N_MELS = 128
N_MFCC = 13

INCLUDE_DELTAS = False # Set True to append Δ and ΔΔ MFCCs

# Optional add-ons
COMPUTE_TEMPO = True
COMPUTE_CHROMA = False # Set True to add key-aware scalar features

TOPK = 5 # Number of neighbors to return

# Evaluation
EVAL_GENRE = True
EVAL_ARTIST = True

In [ ]:
# Preprocessing, feature extraction, aggregation, and I/O

@dataclass
class TrackMeta:
    track_id: str
    title: str
    artist: str
    genre: str
    path: str


def load_tracks_csv(csv_path: str) -> list:
    df = pd.read_csv(csv_path)
    required = {"track_id", "title", "artist", "genre", "path"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"tracks.csv missing columns: {missing}")
    rows = []
    for _, r in df.iterrows():
        rows.append(TrackMeta(
            track_id=str(r["track_id"]),
            title=str(r["title"]),
            artist=str(r["artist"]),
            genre=str(r["genre"]) if not pd.isna(r["genre"]) else "",
            path=str(r["path"]) ))
    return rows


def load_audio_centered(path: str, sr: int = SR, clip_seconds: int = CLIP_SECONDS):
    """Load mono audio and take a centered window of length clip_seconds if audio is longer."""
    y, sr = librosa.load(path, sr=sr, mono=True)
    target_len = clip_seconds * sr
    if len(y) > target_len:
        start = (len(y) - target_len) // 2
        y = y[start:start+target_len]
    y, _ = librosa.effects.trim(y, top_db=30)   # Simple silence trimming at edges
    peak = np.max(np.abs(y)) if len(y) else 1.0   # Peak normalize to prevent loudness dominance
    if peak > 0:
        y = y / peak
    return y, sr


def spectral_features(y: np.ndarray, sr: int):
    S = np.abs(librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH, window=WIN))
    # Power spectrogram for mel and contrast
    S_power = (S ** 2)

    # Spectral stats over time
    centroid = librosa.feature.spectral_centroid(S=S, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(S=S, sr=sr, roll_percent=0.85)
    flatness = librosa.feature.spectral_flatness(S=S)
    zcr = librosa.feature.zero_crossing_rate(y, frame_length=N_FFT, hop_length=HOP_LENGTH)

    feats = {
        "centroid_mean": float(np.mean(centroid)), "centroid_std": float(np.std(centroid)),
        "rolloff_mean": float(np.mean(rolloff)), "rolloff_std": float(np.std(rolloff)),
        "flatness_mean": float(np.mean(flatness)), "flatness_std": float(np.std(flatness)),
        "zcr_mean": float(np.mean(zcr)), "zcr_std": float(np.std(zcr)),
    }

    # Tempo
    if COMPUTE_TEMPO:
        try:
            onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=HOP_LENGTH)
            if onset_env is None or len(onset_env) == 0:
                tempo_val = np.nan
            else:
                tempo_est = librosa.beat.tempo(onset_envelope=onset_env, sr=sr, hop_length=HOP_LENGTH, aggregate=np.median)
                # tempo_est can be a scalar or ndarray([value])
                if isinstance(tempo_est, np.ndarray):
                    tempo_val = float(tempo_est.flat[0]) if tempo_est.size > 0 else np.nan
                else:
                    tempo_val = float(tempo_est)
        except Exception:
              tempo_val = np.nan
        # Ensure feats is a dictionary before adding to it
        if not isinstance(feats, dict):
            feats = {}
        feats["tempo"] = tempo_val

    return feats, S_power

In [ ]:
def extract_features(filepath, COMPUTE_CHROMA=True):
    feats = {}

    y, sr = librosa.load(filepath)
    S = np.abs(librosa.stft(y))
    S_power = S**2

    # Chroma/key
    if COMPUTE_CHROMA:
        try:
            chroma = librosa.feature.chroma_stft(S=S_power, sr=sr)
            chroma_mean = np.mean(chroma, axis=1)
            for i, v in enumerate(chroma_mean):
                feats[f"chroma_mean_{i}"] = float(v)
        except Exception:
            for i in range(12):
                feats[f"chroma_mean_{i}"] = np.nan

    # Clean up NaN/Inf
    for k, v in list(feats.items()):
        if v is None or (isinstance(v, float) and (np.isnan(v) or np.isinf(v))):
            feats[k] = 0.0

    return feats, S_power


In [ ]:
!mkdir -p /content/data
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip -O /content/data/fma_metadata.zip
!unzip -o /content/data/fma_metadata.zip -d /content/data


--2025-10-25 17:29:04--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.16, 2001:620:5ca1:201::214
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘/content/data/fma_metadata.zip’

/content/data/fma_m 100%[===================>] 341.81M  12.2MB/s    in 30s     

2025-10-25 17:29:36 (11.3 MB/s) - ‘/content/data/fma_metadata.zip’ saved [358412441/358412441]

Archive:  /content/data/fma_metadata.zip
 bunzipping: /content/data/fma_metadata/README.txt  
 bunzipping: /content/data/fma_metadata/checksums  
 bunzipping: /content/data/fma_metadata/not_found.pickle  
 bunzipping: /content/data/fma_metadata/raw_genres.csv  
 bunzipping: /content/data/fma_metadata/raw_albums.csv  
 bunzipping: /content/data/fma_metadata/raw_artists.csv  
 bunzipping: /content/data/fma_metada

In [ ]:
!ls -lh /content/data/fma_metadata/tracks.csv | grep tracks.csv


-r--r--r-- 1 root root 249M Apr  1  2017 /content/data/fma_metadata/tracks.csv


In [ ]:
TRACKS_CSV = "/content/data/fma_metadata/tracks.csv"
metas = load_tracks_csv(TRACKS_CSV)
metas_existing = [m for m in metas if os.path.exists(m["path"])]

In [ ]:
DATA_DIR = "/content/data"
os.makedirs(DATA_DIR, exist_ok=True)
FEATURES_CSV = os.path.join(DATA_DIR, "features_parallel.csv")

In [ ]:
FEATURES_CSV = "/content/data/features_parallel.csv"  # or your actual path
feat_df = pd.read_csv(FEATURES_CSV)
print(feat_df.shape)

(1000, 40)


In [ ]:
# MFCC FEATURES

from typing import Dict, Optional, Any

def _mget(meta, key, default=None):
    """Helper to get key from dict or attribute from object."""
    if isinstance(meta, dict):
        return meta.get(key, default)
    return getattr(meta, key, default)


def mfcc_features(S_power: np.ndarray, sr: int) -> Dict[str, float]:
    """
    Compute MFCC stats from a power spectrogram S_power (|STFT|^2).
    """
    # Mel-spectrogram (power)
    mel = librosa.feature.melspectrogram(S=S_power, sr=sr, n_mels=N_MELS)

    # Log-mel in dB
    log_mel = librosa.power_to_db(mel + 1e-10)

    # MFCC directly from log-mel
    mfcc = librosa.feature.mfcc(S=log_mel, sr=sr, n_mfcc=N_MFCC)

    feats: Dict[str, float] = {}

    # Aggregate mean and std across time for each coefficient
    for i in range(N_MFCC):
        coeff = mfcc[i]
        feats[f"mfcc_{i+1}_mean"] = float(np.nanmean(coeff))
        feats[f"mfcc_{i+1}_std"]  = float(np.nanstd(coeff))

    if INCLUDE_DELTAS:
        d1 = librosa.feature.delta(mfcc)
        d2 = librosa.feature.delta(mfcc, order=2)
        for i in range(N_MFCC):
            feats[f"mfcc_delta_{i+1}_mean"]       = float(np.nanmean(d1[i]))
            feats[f"mfcc_delta_{i+1}_std"]        = float(np.nanstd(d1[i]))
            feats[f"mfcc_deltadelta_{i+1}_mean"]  = float(np.nanmean(d2[i]))
            feats[f"mfcc_deltadelta_{i+1}_std"]   = float(np.nanstd(d2[i]))

    # Final cleanup: replace NaN/Inf with 0.0
    for k, v in list(feats.items()):
        if v is None or (isinstance(v, float) and (np.isnan(v) or np.isinf(v))):
            feats[k] = 0.0

    return feats

def extract_features_for_track(meta: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    try:
        # Load a centered clip
        path = _mget(meta, "path")
        if not path:
            raise ValueError("Missing audio path in meta")
        if not os.path.exists(path):
            raise FileNotFoundError(f"Audio not found at {path}")

        y, sr = load_audio_centered(path, sr=SR, clip_seconds=CLIP_SECONDS)

        # Your spectral features function should return (dict, S_power)
        spec_feats, S_power = spectral_features(y, sr)

        # MFCC features from S_power
        mfcc_feats = mfcc_features(S_power, sr)

        # Merge and attach metadata
        feats: Dict[str, Any] = {**spec_feats, **mfcc_feats}
        feats.update({
            "track_id": _mget(meta, "track_id"),
            "title": _mget(meta, "title"),
            "artist": _mget(meta, "artist"),
            "genre": _mget(meta, "genre"),
            "path": path,
        })
        return feats

    except Exception as e:
        print(f"[WARN] Failed {_mget(meta, 'track_id', 'unknown')}: {e}")
        return None

In [ ]:
# MFCC FEATURES

from typing import Dict, Optional, Any

def extract_features_for_track(meta: Dict[str, Any]) -> Optional[Dict[str, Any]]:

    def _mget(meta, key, default=None):
      """Helper to get key from dict or attribute from object."""
      if isinstance(meta, dict):
          return meta.get(key, default)
      return getattr(meta, key, default)


def mfcc_features(S_power: np.ndarray, sr: int) -> Dict[str, float]:
    """
    Compute MFCC stats from a power spectrogram S_power (|STFT|^2).
    """
    # Mel-spectrogram (power)
    mel = librosa.feature.melspectrogram(S=S_power, sr=sr, n_mels=N_MELS)

    # Log-mel in dB
    log_mel = librosa.power_to_db(mel + 1e-10)

    # MFCC directly from log-mel
    mfcc = librosa.feature.mfcc(S=log_mel, sr=sr, n_mfcc=N_MFCC)

    feats: Dict[str, float] = {}

    # Aggregate mean and std across time for each coefficient
    for i in range(N_MFCC):
        coeff = mfcc[i]
        feats[f"mfcc_{i+1}_mean"] = float(np.nanmean(coeff))
        feats[f"mfcc_{i+1}_std"]  = float(np.nanstd(coeff))

    if INCLUDE_DELTAS:
        d1 = librosa.feature.delta(mfcc)
        d2 = librosa.feature.delta(mfcc, order=2)
        for i in range(N_MFCC):
            feats[f"mfcc_delta_{i+1}_mean"]       = float(np.nanmean(d1[i]))
            feats[f"mfcc_delta_{i+1}_std"]        = float(np.nanstd(d1[i]))
            feats[f"mfcc_deltadelta_{i+1}_mean"]  = float(np.nanmean(d2[i]))
            feats[f"mfcc_deltadelta_{i+1}_std"]   = float(np.nanstd(d2[i]))

    # Final cleanup: replace NaN/Inf with 0.0
    for k, v in list(feats.items()):
        if v is None or (isinstance(v, float) and (np.isnan(v) or np.isinf(v))):
            feats[k] = 0.0

    return feats

def extract_features_for_track(meta: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    try:
        # Load a centered clip
        path = _mget(meta, "path")
        if not path:
            raise ValueError("Missing audio path in meta")
        if not os.path.exists(path):
            raise FileNotFoundError(f"Audio not found at {path}")

        y, sr = load_audio_centered(path, sr=SR, clip_seconds=CLIP_SECONDS)

        # Your spectral features function should return (dict, S_power)
        spec_feats, S_power = spectral_features(y, sr)

        # MFCC features from S_power
        mfcc_feats = mfcc_features(S_power, sr)

        # Merge and attach metadata
        feats: Dict[str, Any] = {**spec_feats, **mfcc_feats}
        feats.update({
            "track_id": _mget(meta, "track_id"),
            "title": _mget(meta, "title"),
            "artist": _mget(meta, "artist"),
            "genre": _mget(meta, "genre"),
            "path": path,
        })
        return feats

    except Exception as e:
        print(f"[WARN] Failed {_mget(meta, 'track_id', 'unknown')}: {e}")
        return None

In [ ]:
# Feature Table

AUDIO_ROOT = "/content/fma_small"

def load_tracks_csv(csv_path: str):
    # Load multi-index CSV
    df = pd.read_csv(csv_path, header=[0, 1], index_col=0)

    rows = []
    for track_id, row in df.iterrows():
        title = row[('track', 'title')]
        genre = row[('track', 'genre_top')]
        artist = row[('artist', 'name')]

        # Build path to audio file for FMA small
        track_id_int = int(track_id)
        folder = f"{track_id_int:03d}"[:3]
        filename = f"{track_id_int:06d}.mp3"
        audio_path = os.path.join("/content/fma_small", folder, filename)

        rows.append({
            "track_id": track_id,
            "title": title,
            "artist": artist,
            "genre": genre,
            "path": audio_path
        })
    return rows

    TRACKS_CSV = "/content/data/fma_metadata/tracks.csv"
metas = load_tracks_csv(TRACKS_CSV)
print(f"Loaded {len(metas)} tracks.")
print(metas[0])

Loaded 106574 tracks.
{'track_id': 2, 'title': 'Food', 'artist': 'AWOL', 'genre': 'Hip-Hop', 'path': '/content/fma_small/002/000002.mp3'}


In [ ]:
AUDIO_ROOT = "/content/fma_small"

def build_fma_small_path(track_id: int, audio_root=AUDIO_ROOT) -> str:
    tid = int(track_id)
    folder = f"{tid:06d}"[:3]
    filename = f"{tid:06d}.mp3"
    return os.path.join(audio_root, folder, filename)


for m in metas:
    m["path"] = build_fma_small_path(m["track_id"])

# Filter to tracks whose audio actually exists
metas_existing = [m for m in metas if os.path.exists(m["path"])]

print(f"Total metadata rows: {len(metas)}")
print(f"Audio files found:   {len(metas_existing)} (should be ~8000 for fma_small)")
print(metas_existing[0] if metas_existing else "No audio files found; check unzip path.")

Total metadata rows: 106574
Audio files found:   8000 (should be ~8000 for fma_small)
{'track_id': 2, 'title': 'Food', 'artist': 'AWOL', 'genre': 'Hip-Hop', 'path': '/content/fma_small/000/000002.mp3'}


In [ ]:
!pip -q install joblib

import os, pandas as pd, multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

def process_one(meta):
    return extract_features_for_track(meta)  # already defined earlier

def run_parallel(metas, n_jobs=None, batch_size=100, out_csv=None):
    """
    Parallel feature extraction with joblib and periodic CSV saving.
    """
    if n_jobs is None:
        n_jobs = max(1, multiprocessing.cpu_count() - 1)

    rows = []
    for start in tqdm(range(0, len(metas), batch_size), desc="Extracting (parallel)"):
        chunk = metas[start:start+batch_size]
        results = Parallel(n_jobs=n_jobs, backend="loky", prefer="processes")(
            delayed(process_one)(m) for m in chunk
        )
        good = [r for r in results if r is not None]
        rows.extend(good)

        # Save partial results
        if out_csv and len(good) > 0:
            mode = "a" if os.path.exists(out_csv) else "w"
            header = not os.path.exists(out_csv)
            pd.DataFrame(good).to_csv(out_csv, mode=mode, header=header, index=False)
            rows = []

    return rows

In [ ]:
rows = run_parallel(metas_existing, n_jobs=None, batch_size=100, out_csv=FEATURES_CSV)
feat_df = pd.DataFrame(rows)

Extracting (parallel):  10%|█         | 8/80 [05:57<53:40, 44.72s/it]


KeyboardInterrupt: 

In [ ]:
def _mget(meta, key, default=None):

    if isinstance(meta, dict):
        return meta.get(key, default)
    return getattr(meta, key, default)

def extract_features_for_track(meta):
    try:
        path = _mget(meta, "path")
        if not path:
            raise ValueError("Missing audio path in meta")
        if not os.path.exists(path):
            raise FileNotFoundError(f"Audio not found at {path}")

        # Load audio
        y, sr = load_audio_centered(path, sr=SR, clip_seconds=CLIP_SECONDS)

        # Feature blocks
        spec_feats, S_power = spectral_features(y, sr)
        mfcc_feats = mfcc_features(S_power, sr)

        # Merge + attach metadata
        feats = {**spec_feats, **mfcc_feats}
        feats.update({
            "track_id": _mget(meta, "track_id"),
            "title": _mget(meta, "title"),
            "artist": _mget(meta, "artist"),
            "genre": _mget(meta, "genre"),
            "path": path,
        })
        return feats

    except Exception as e:
        tid = _mget(meta, "track_id", "unknown")
        print(f"[WARN] Failed {tid}: {e}")
        return None

rows = []
for m in tqdm(metas_existing, desc="Extracting features"):
    feats = extract_features_for_track(m)
    if feats is not None:
        rows.append(feats)

feat_df = pd.DataFrame(rows)
print(f"Tracks with features: {len(feat_df)}, Columns: {feat_df.shape[1] if len(feat_df)>0 else 0}")

Extracting features:   0%|          | 0/8000 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Standardize + Similarity Index

# Select numeric columns for modeling
meta_cols = ["track_id", "title", "artist", "genre", "path"]
num_cols = [c for c in feat_df.columns if c not in meta_cols]

# Handle any NaNs
feat_df[num_cols] = feat_df[num_cols].replace([np.inf, -np.inf], np.nan).fillna(feat_df[num_cols].mean())

scaler = StandardScaler()
X = scaler.fit_transform(feat_df[num_cols].values)

# Precompute cosine similarity matrix
sim_mat = cosine_similarity(X)
np.fill_diagonal(sim_mat, 0.0) # ignore self-similarity in retrieval

In [ ]:
# Top-K Neighbors for given index

TOPK = 10

# Build lookup tables
idx2meta = {i: m for i, m in enumerate(metas_existing)}
title_artist2idx = {(m['title'].strip().lower(), m['artist'].strip().lower()): i
                    for i, m in enumerate(metas_existing)}

def topk_for_index(idx: int, k: int = TOPK):
    sims = sim_mat[idx]
    nn_idx = np.argsort(-sims)[:k]
    return [(int(j), float(sims[j])) for j in nn_idx]

# spot-check
for idx in [0, min(1, len(feat_df)-1)]:
    seed = feat_df.iloc[idx]
    nns = topk_for_index(idx, k=TOPK)

    print(f"\nSeed: {seed['title']} — {seed['artist']}")

    # Convert neighbors to table rows
    table = []
    for rank, (j, s) in enumerate(nns, start=1):
        row = feat_df.iloc[j]
        table.append({
            "Rank": rank,
            "Track ID": j,
            "Title": row['title'],
            "Artist": row['artist'],
            "Genre": row.get('genre', 'Unknown'),
            "Similarity": round(s, 3)
        })

    df_neighbors = pd.DataFrame(table)
    display(df_neighbors)


Seed: Food — AWOL


,Rank,Track ID,Title,Artist,Genre,Similarity
0,1,400,Food,AWOL,Hip-Hop,1.000
1,2,300,Food,AWOL,Hip-Hop,1.000
2,3,800,Food,AWOL,Hip-Hop,1.000
3,4,337,Planet Say (featuring Faust),Food For Animals,Hip-Hop,0.868
4,5,837,Planet Say (featuring Faust),Food For Animals,Hip-Hop,0.868
5,6,37,Planet Say (featuring Faust),Food For Animals,Hip-Hop,0.868
6,7,437,Planet Say (featuring Faust),Food For Animals,Hip-Hop,0.868
7,8,717,2009,Height With Friends,Hip-Hop,0.865
8,9,718,Clack Struck Twelve,Height With Friends,Hip-Hop,0.845
9,10,761,broke Pimpin,BrokeMC,Hip-Hop,0.824



Seed: This World — AWOL


,Rank,Track ID,Title,Artist,Genre,Similarity
0,1,401,This World,AWOL,Hip-Hop,1.000
1,2,801,This World,AWOL,Hip-Hop,1.000
2,3,301,This World,AWOL,Hip-Hop,1.000
3,4,761,broke Pimpin,BrokeMC,Hip-Hop,0.826
4,5,712,The Woods,Height With Friends,Hip-Hop,0.807
5,6,716,Cavalcade Lagoons,Height With Friends,Hip-Hop,0.799
6,7,449,Relaxing,Glove Compartment,Rock,0.792
7,8,49,Relaxing,Glove Compartment,Rock,0.792
8,9,849,Relaxing,Glove Compartment,Rock,0.792
9,10,349,Relaxing,Glove Compartment,Rock,0.792


In [ ]:
# Evaluation
# Implement: Feature Stability, Artist@K, Genre Purity@K, and MRR for same-artist positives

def _mget(meta, key, default=None):
    return meta.get(key, default) if isinstance(meta, dict) else getattr(meta, key, default)

def _mfcc_means_from_segment(y, sr, seg_seconds=5, n_mfcc=20):
    seg_len = int(seg_seconds * sr)
    if len(y) < seg_len + 1:
        return None
    start = np.random.randint(0, max(1, len(y) - seg_len))
    seg = y[start:start + seg_len]

    S_power = np.abs(librosa.stft(seg))**2
    mel = librosa.feature.melspectrogram(S=S_power, sr=sr, n_mels=128)
    log_mel = librosa.power_to_db(mel + 1e-10)
    mfcc = librosa.feature.mfcc(S=log_mel, sr=sr, n_mfcc=n_mfcc)
    return np.nanmean(mfcc, axis=1)

def feature_stability_v2(meta, trials: int = 2):
    """Correlation between two MFCC-mean vectors from different segments of the same track."""
    path = _mget(meta, "path")
    if not path:
        raise ValueError("Meta missing 'path'")
    y, sr = load_audio_centered(path, sr=SR, clip_seconds=CLIP_SECONDS)
    if y is None or len(y) < 4 * sr:
        return np.nan

    x1 = _mfcc_means_from_segment(y, sr)
    x2 = _mfcc_means_from_segment(y, sr)
    if x1 is None or x2 is None:
        return np.nan

    if np.allclose(np.std(x1), 0) or np.allclose(np.std(x2), 0):
        return np.nan
    return float(np.corrcoef(x1, x2)[0, 1])


def evaluate_artist_at_k(k: int = TOPK):
    if not EVAL_ARTIST:
        return None
    hits = 0
    denom = 0
    for i in range(len(feat_df)):
        seed_artist = str(feat_df.iloc[i]["artist"]).strip().lower()
        if len(seed_artist) == 0:
            continue
        denom += 1
        nns = topk_for_index(i, k)
        found = False
        for j, _ in nns:
            if str(feat_df.iloc[j]["artist"]).strip().lower() == seed_artist:
                found = True
                break
        hits += int(found)
    return hits / denom if denom > 0 else np.nan

def evaluate_genre_purity_at_k(k: int = TOPK):
    if not EVAL_GENRE:
        return None
    purities = []
    for i in range(len(feat_df)):
        seed_genre = str(feat_df.iloc[i]["genre"]).strip().lower()
        if len(seed_genre) == 0:
            continue
        nns = topk_for_index(i, k)
        same = 0
        for j, _ in nns:
            if str(feat_df.iloc[j]["genre"]).strip().lower() == seed_genre:
               same += 1
        purities.append(same / k)
    return float(np.mean(purities)) if purities else np.nan


def evaluate_mrr_same_artist(k: int = TOPK):
    ranks = []
    for i in range(len(feat_df)):
        seed_artist = str(feat_df.iloc[i]["artist"]).strip().lower()
        if len(seed_artist) == 0:
            continue
        sims = sim_mat[i]
        order = np.argsort(-sims)
        # find first index with same artist
        rr = 0.0
        for rank, j in enumerate(order[:max(50, k)], start=1):
            if str(feat_df.iloc[j]["artist"]).strip().lower() == seed_artist:
                rr = 1.0 / rank
                break
        if rr > 0:
            ranks.append(rr)
        return float(np.mean(ranks)) if ranks else np.nan


In [ ]:
print(type(metas_existing[0]), list(metas_existing[0].keys()))
print("Sample path:", metas_existing[0]["path"])
print("One stability value (v2):", feature_stability_v2(metas_existing[0], trials=2))

<class 'dict'> ['track_id', 'title', 'artist', 'genre', 'path']
Sample path: /content/fma_small/000/000002.mp3
One stability value (v2): 0.9990657080071833


In [ ]:
try:
    del feature_stability #remove previous version from memeory
except NameError:
    pass
feature_stability = feature_stability_v2  # point the name to the new dict


In [ ]:
random.seed(42)

EVAL_SIZE = 500
metas_eval = random.sample(metas_existing, EVAL_SIZE)  # subset of dicts

# map the subset back to their row indices (to use with sim_mat)
meta_to_idx = {id(m): i for i, m in enumerate(metas_existing)}
seed_indices = [meta_to_idx[id(m)] for m in metas_eval]
print(f"Using {len(metas_eval)} tracks for evaluation")

Using 500 tracks for evaluation


In [ ]:
# Stability across tracks

stabs = []
for m in tqdm(metas_eval, desc="Stability (MFCC means corr)"):
    s = feature_stability(m, trials=2)
    try:
        stabs.append(float(s))
    except Exception:
        stabs.append(np.nan)

stab_arr = np.asarray(stabs, dtype=float)
stability_mean = float(np.nanmean(stab_arr)) if np.isfinite(stab_arr).any() else np.nan

artist_at_k  = evaluate_artist_at_k(k=TOPK)  # if needed, can later adapt to subset
genre_purity = evaluate_genre_purity_at_k(k=TOPK)
mrr          = evaluate_mrr_same_artist(k=TOPK)

def fmt(x):
    return "nan" if not (isinstance(x, (int, float)) and np.isfinite(x)) else f"{x:.3f}"

print("\n=== Evaluation ===")
print(f"Feature Stability (MFCC means corr): {fmt(stability_mean)}")
print(f"Artist@{TOPK}: {fmt(artist_at_k)}")
print(f"Genre Purity@{TOPK}: {fmt(genre_purity)}")
print(f"MRR (same-artist within top ~50): {fmt(mrr)}")


Stability (MFCC means corr): 100%|██████████| 500/500 [02:02<00:00,  4.09it/s]



=== Evaluation ===
Feature Stability (MFCC means corr): 0.973
Artist@10: 0.949
Genre Purity@10: 0.648
MRR (same-artist within top ~50): 1.000


In [ ]:
# Identify columns
mfcc_cols = [c for c in feat_df.columns if c.startswith("mfcc_")]
spectral_cols = [c for c in feat_df.columns if c not in meta_cols + mfcc_cols]

# MFCC-only pipeline
X_mfcc = StandardScaler().fit_transform(feat_df[mfcc_cols].values)
sim_mfcc = cosine_similarity(X_mfcc)
np.fill_diagonal(sim_mfcc, 0.0)

In [ ]:
# Evaluate Artist@K for a sim matrix

def eval_artist_at_k_with_sim(sim_matrix, k: int = TOPK):
  hits = 0
  denom = 0
  for i in range(len(feat_df)):
      seed_artist = str(feat_df.iloc[i]["artist"]).strip().lower()
      if len(seed_artist) == 0:
          continue
      denom += 1
      sims = sim_matrix[i]
      order = np.argsort(-sims)[:k]
      found = False
      for j in order:
          if str(feat_df.iloc[j]["artist"]).strip().lower() == seed_artist:
              found = True
              break
      hits += int(found)
  return hits / denom if denom > 0 else np.nan

artist_at_k_mfcc = eval_artist_at_k_with_sim(sim_mfcc, k=TOPK)
artist_at_k_combined = evaluate_artist_at_k(k=TOPK)

print("\n=== Ablation (Artist@K) ===")
print(f"MFCC-only: {artist_at_k_mfcc:.3f}")
print(f"MFCC+Spectral: {artist_at_k_combined:.3f}")


=== Ablation (Artist@K) ===
MFCC-only: 0.951
MFCC+Spectral: 0.949


In [ ]:
# Retrieval API for report

INDEX_BY_ID = {tid: i for i, tid in enumerate(feat_df["track_id"]) }

def recommend_by_track_id(track_id: str, k: int = TOPK):
    idx = INDEX_BY_ID.get(track_id, None)
    if idx is None:
        raise KeyError(f"Unknown track_id: {track_id}")
    nns = topk_for_index(idx, k=k)
    rows = []
    seed = feat_df.iloc[idx]
    for j, s in nns:
        row = feat_df.iloc[j]
        rows.append({
            "seed_track_id": seed["track_id"],
            "seed_title": seed["title"],
            "seed_artist": seed["artist"],
            "rec_track_id": row["track_id"],
            "rec_title": row["title"],
            "rec_artist": row["artist"],
            "similarity": round(float(s), 4),
            })
    df = pd.DataFrame(rows)
    print(f"✅ Top {k} recommendations for track ID {track_id} — {seed['title']} by {seed['artist']}")
    return df

In [ ]:
some_id = int(feat_df["track_id"].iloc[0])
recommend_by_track_id(some_id, k=5)

✅ Top 5 recommendations for track ID 2 — Food by AWOL


,seed_track_id,seed_title,seed_artist,rec_track_id,rec_title,rec_artist,similarity
0,2,Food,AWOL,2,Food,AWOL,1.0000
1,2,Food,AWOL,2,Food,AWOL,1.0000
2,2,Food,AWOL,2,Food,AWOL,1.0000
3,2,Food,AWOL,694,Planet Say (featuring Faust),Food For Animals,0.8677
4,2,Food,AWOL,694,Planet Say (featuring Faust),Food For Animals,0.8677
